# Data cleaning + feature engineering on full data

## Imports

In [1]:
import os
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

In [2]:
os.getcwd()

'/home/ec2-user/notebooks'

In [3]:
info_file = '/home/ec2-user/film-data/raw-data/info.csv'
credits_file = '/home/ec2-user/film-data/raw-data/credits.csv'
releases_file = '/home/ec2-user/film-data/raw-data/releases.csv'

In [4]:
#info table contains basic film details like genre, budget, revenue, language, runtime, etc.
info_df = pd.read_csv(info_file)

#credits table contains cast and crew
credits_df = pd.read_csv(credits_file)

#releases table contains info on which countries each film was released in
releases_df = pd.read_csv(releases_file)

/home/ec2-user/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,3,6,11,16,17,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ec2-user/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Cleaning + new features

#### Because of the way data was collected, last row of each table has the headers. Fixing that:

In [5]:
info_df.columns = info_df.iloc[-1:, :].squeeze()
info_df = info_df.iloc[:-1, :]

In [6]:
credits_df.columns = credits_df.iloc[-1:, :].squeeze()
credits_df = credits_df.iloc[:-1, :]

In [7]:
releases_df.columns = releases_df.iloc[-1:, :].squeeze()
releases_df = releases_df.iloc[:-1, :]

### Cleaning info table

#### Getting each film's genre:

In [8]:
def get_first_genre(lod_str):
    '''
    Function for use with pandas apply method.
    --------
    INPUT
    lod_str (str): a list of dictionaries in the form of a string
    --------
    OUTPUT
    (str): name of the first genre from the list of dicts
    '''
    lst_of_dicts = eval(lod_str)
    if len(lst_of_dicts) > 0:
        for i in range(len(lst_of_dicts)):
            return lst_of_dicts[i]['name']
    else:
        pass

In [9]:
info_df['genre'] = info_df['genres'].apply(get_first_genre)  #func in cell above

#### Dropping unnecessary rows and columns from info table:

In [10]:
#making sure relevant numerical columns are int/float type
info_df = info_df.astype({"adult": str, 
                          "budget": float, 
                          "id": int, 
                          "revenue": float, 
                          "original_language": str, 
                          "runtime": float, 
                          "genre": str})

In [11]:
#filtering for only non-adult films,
# with revenue & budget > 0,
# original language english,
# runtime >= 80 minutes,
# and genre is not documentary or tv movie.

info_df = info_df[((info_df['adult'] == 'False') | (info_df['adult'] == False)) & \
        (info_df['budget'] > 0) & \
        (info_df['revenue'] > 0) & \
        (info_df['original_language'] == 'en') & \
        (info_df['runtime'] >= 80.0) & \
        (info_df['genre'] != 'Documentary') & (info_df['genre'] != 'TV Movie')]

In [12]:
#dropping columns from info_df that won't be needed for EDA
drop_cols = ['adult', 'backdrop_path', 'belongs_to_collection', 
             'genres', 'homepage', 'imdb_id', 'original_language', 'original_title', 
             'overview', 'popularity', 'poster_path', 'production_companies', 
             'production_countries', 'spoken_languages', 'status', 'tagline', 'video', 
             'vote_average', 'vote_count']

info_df = info_df.drop(drop_cols, axis=1)

### a few more cols to be dropped later for modelling, after EDA, feat. engineering, & joins: 
### ['id', 'release_date', 'runtime' (possibly), 'title', 'genre' (possibly)]

#### Creating columns for (a) amount of profit or loss (revenue minus budget) and (b) whether each movie profited (binary, whether revenue > budget):

In [13]:
info_df['profit'] = info_df['revenue'] - info_df['budget']

In [14]:
info_df['made_money'] = info_df['profit'] > 0
info_df['made_money'] = info_df['made_money'].astype('int')

In [15]:
#converting release date column from object type to datetime type
info_df['release_date'] = pd.to_datetime(info_df['release_date'])

### Joining the tables

#### Note: when the keys to join on are pd columns, not indices, better to use merge method

In [16]:
#making sure id column is int so dataframes can be merged
credits_df = credits_df.astype({"id": int})

In [17]:
#making sure id column is int so dataframes can be merged
releases_df = releases_df.astype({"id": int})

In [18]:
step1 = info_df.merge(credits_df, on='id', how='inner')

In [19]:
df = step1.merge(releases_df, on='id', how='inner')

### Cleaning credits table

#### Appending columns with top billed actors to the credits table:

In [20]:
def get_top_cast(df, lod_col, billings):
    '''
    INPUT
    df: pandas dataframe w/ stringified lists of dictionaries containing cast
    lod_col (str): name of col w/ stringified lists of dictionaries
    billings (int): number of cast members to extract from each lod
    --------
    OUTPUT
    original df w/ new columns having actor names corresponding to each billing 
    number in this function (lower billing number = more prominent
    role; 0 = star of the movie, 1 = costar, etc.)
    '''
    for c in range(billings):
        billing = c
        for i in range(len(df)):
            lst_of_dicts = eval(df.at[i, lod_col]) #lod's are strings at first, hence eval
            if len(lst_of_dicts) > billing:
                dict_of_interest = lst_of_dicts[billing]
                actor = dict_of_interest['name']
                df.at[i, 'cast_'+str(billing)] = actor
            else:
                continue
    return df

In [21]:
df = get_top_cast(df, 'cast', 5)  #func in cell above

#### Appending columns with director and screenwriters to the credits table:

In [22]:
def get_director(lod_str):
    '''
    Function for use with pandas apply method.
    --------
    INPUT
    lod_str (str): a list of dictionaries in the form of a string
    --------
    OUTPUT
    (str): name of the first director from the list of dicts
    '''
    lst_of_dicts = eval(lod_str)
    if len(lst_of_dicts) > 0:
        for i in range(len(lst_of_dicts)):
            if lst_of_dicts[i]['job'] == 'Director':
                return lst_of_dicts[i]['name']
    else:
        pass

In [23]:
df['director'] = df['crew'].apply(get_director)  #func in cell above

In [24]:
def get_first_writer(lod_str):
    '''
    Function for use with pandas apply method.
    --------
    INPUT
    lod_str (str): a list of dictionaries in the form of a string
    --------
    OUTPUT
    (str): name of the first screenwriter from the list of dicts
    '''
    lst_of_dicts = eval(lod_str)
    if len(lst_of_dicts) > 0:
        for i in range(len(lst_of_dicts)):
            if lst_of_dicts[i]['job'] == 'Screenplay' or \
            lst_of_dicts[i]['job'] == 'Writer' or \
            lst_of_dicts[i]['job'] == 'Author':
                return lst_of_dicts[i]['name']
    else:
        pass

In [25]:
df['writer'] = df['crew'].apply(get_first_writer)  #func in cell above

In [26]:
#decided not to get second writer becuse relatively few films have one in the dataset

In [27]:
creds_drop_cols = ['cast', 'crew']
df = df.drop(creds_drop_cols, axis=1)

#### Dropping rows without at least 5 actors, or missing director/writer/release:

In [33]:
df = df.dropna(subset=['cast_4', 'director', 'writer', 'release_date'])

### Cleaning releases table

#### Getting number of countries each film was released in and dropping nulls:

In [37]:
def get_releases_count(lod_str):
    '''
    Function for use with pandas apply method.
    --------
    INPUT
    lod_str (str): a list of dictionaries in the form of a string
    --------
    OUTPUT
    (int): number of countries/territories the film was released in
    '''
    lst_of_dicts = eval(lod_str)
    if len(lst_of_dicts) > 0:
        return len(lst_of_dicts)
    else:
        pass

In [38]:
df['releases'] = df['countries'].apply(get_releases_count)  #func in cell above

In [39]:
df = df.drop('countries', axis=1)

### Checking for outliers/faulty data

In [42]:
pd.set_option('float_format', '{:f}'.format)  #so that floats will print w/o scientific notat.

In [61]:
df.describe(); #checking for outliers/faulty data

In [62]:
df = df[df['budget'] > 10000] #dropping microbudget films and faulty data

In [63]:
df = df[df['revenue'] > 10000] #dropping microbudget films and faulty data

In [67]:
df.describe(); #much better

In [68]:
# resetting index for easier for-looping later
df = df.reset_index().drop('index', axis=1)

### Getting cast & crew lifetime revenue, profits, # of films, # of profitable films

In [71]:
def get_cast_metrics(df, cast_cols):
    '''
    Iterates over columns of cast members
    and generates new columns with the total
    revenue, profit, number of movies, and number of
    profitable movies that cast collectively has been in
    up until the date of release in each row.
    --------
    INPUT
    df: dataframe with cast columns
    cast_cols: list of column names (str) with the cast
    '''
    rev_cols = []
    prof_cols = []
    prof_count_cols = []
    tot_count_cols = []
    for col_name in cast_cols:
        for i in range(len(df)):
            #1. getting a dataframe with actor's movies to date only
            name = df.at[i, col_name]
            release_date = df.at[i, 'release_date']
            df_to_sum = df[(df['release_date'] < release_date) & \
                           ((df['cast_0'] == name) | (df['cast_1'] == name) | \
                            (df['cast_2'] == name) | (df['cast_3'] == name) | \
                            (df['cast_4'] == name))]
            #2. creating new cols for each actor's lifetime rev, prof, etc. to date
            df.at[i, str(col_name)+'_rev'] = df_to_sum['revenue'].sum()
            df.at[i, str(col_name)+'_prof'] = df_to_sum['profit'].sum()
            df.at[i, str(col_name)+'_films'] = df_to_sum['made_money'].count()
            df.at[i, str(col_name)+'_prof_films'] = df_to_sum['made_money'].sum()
        
        #2.1 appending those new col names to a list for subsequent summing and dropping
        rev_cols.append(str(col_name)+'_rev')
        prof_cols.append(str(col_name)+'_prof')
        prof_count_cols.append(str(col_name)+'_prof_films')
        tot_count_cols.append(str(col_name)+'_films')

    #3. summing/dropping those cols so we just have cols representing top5 cast as a whole
    
    rev_to_sum = df[rev_cols].copy()
    prof_to_sum = df[prof_cols].copy()
    prof_count_to_sum = df[prof_count_cols].copy()
    tot_count_to_sum = df[tot_count_cols].copy()
    
    df['cast_rev'] = rev_to_sum.sum(axis=1)
    df['cast_prof'] = prof_to_sum.sum(axis=1)
    df['cast_films'] = tot_count_to_sum.sum(axis=1)
    df['cast_prof_films'] = prof_count_to_sum.sum(axis=1)
    
    df = df.drop(rev_cols, axis=1)
    df = df.drop(prof_cols, axis=1)
    df = df.drop(prof_count_cols, axis=1)
    df = df.drop(tot_count_cols, axis=1)
    
    return df

In [72]:
cast_cols = ['cast_0', 'cast_1', 'cast_2', 'cast_3', 'cast_4']

#this may take a while to run, reason to use Spark on full data:
df = get_cast_metrics(df, cast_cols)  #func from cell above

In [73]:
def get_director_metrics(df, director_col):
    '''
    Iterates over column with director
    and generates new columns with the total
    revenue, profit, number of movies, and number of
    profitable movies the director has been in
    up until the date of release for each row.
    --------
    INPUT
    df: dataframe with director columns
    director_col: column name (str) with director
    '''
    for i in range(len(df)):
        #1. getting a dataframe with director's movies to date only
        name = df.at[i, director_col]
        release_date = df.at[i, 'release_date']
        df_to_sum = df[(df['release_date'] < release_date) & (df[director_col] == name)]
        
        #2. creating new cols for each director's lifetime rev, prof, etc. to date
        df.at[i, 'dir_rev'] = df_to_sum['revenue'].sum()
        df.at[i, 'dir_prof'] = df_to_sum['profit'].sum()
        df.at[i, 'dir_films'] = df_to_sum['made_money'].count()
        df.at[i, 'dir_prof_films'] = df_to_sum['made_money'].sum()
    
    return df

In [75]:
df = get_director_metrics(df, 'director')  #func from cell above

In [76]:
def get_writer_metrics(df, writer_col):
    '''
    Iterates over column with writer
    and generates new columns with the total
    revenue, profit, number of movies, and number of
    profitable movies the writer has been in
    up until the date of release for each row.
    --------
    INPUT
    df: dataframe with writer columns
    writer_col: column name (str) with writer
    '''
    for i in range(len(df)):
        #1. getting a dataframe with writer's movies to date only
        name = df.at[i, writer_col]
        release_date = df.at[i, 'release_date']
        df_to_sum = df[(df['release_date'] < release_date) & (df[writer_col] == name)]
        
        #2. creating new cols for each writer's lifetime rev, prof, etc. to date
        df.at[i, 'writ_rev'] = df_to_sum['revenue'].sum()
        df.at[i, 'writ_prof'] = df_to_sum['profit'].sum()
        df.at[i, 'writ_films'] = df_to_sum['made_money'].count()
        df.at[i, 'writ_prof_films'] = df_to_sum['made_money'].sum()
    
    return df

In [77]:
df = get_writer_metrics(df, 'writer')  #func from cell above

### Getting average of cast & crew revenue, profit, etc. for each film's competitors (i.e. all other films released with +/- 2 weeks of each film):

In [80]:
def get_competitor_metrics(df, id_col):
    '''
    Iterates over each film (each row)
    and generates new columns with the average
    revenue, profit, number of movies, and number of
    profitable movies of the casts/crews from competing movies
    (i.e. those released within +/- 2 weeks) have been in.
    --------
    INPUT
    df: dataframe with cast columns
    id_col: column name (str) with film id
    '''
    for i in range(len(df)):
        #1. getting a dataframe with competing films
        id_ = df.at[i, id_col]
        release_date = df.at[i, 'release_date']
        df_to_avg = df[(df['release_date'] >= (release_date - pd.to_timedelta(14, unit='d'))) & \
               (df['release_date'] <= (release_date + pd.to_timedelta(14, unit='d'))) & \
               (df['id'] != id_)]
        
        #2. creating new cols for competing cast/crews' avg lifetime rev, prof, etc.
        df.at[i, 'compet_cast_rev'] = df_to_avg['cast_rev'].mean()
        df.at[i, 'compet_cast_prof'] = df_to_avg['cast_prof'].mean()
        df.at[i, 'compet_cast_films'] = df_to_avg['cast_films'].mean()
        df.at[i, 'compet_cast_prof_films'] = df_to_avg['cast_prof_films'].mean()
        
        df.at[i, 'compet_dir_rev'] = df_to_avg['dir_rev'].mean()
        df.at[i, 'compet_dir_prof'] = df_to_avg['dir_prof'].mean()
        df.at[i, 'compet_dir_films'] = df_to_avg['dir_films'].mean()
        df.at[i, 'compet_dir_prof_films'] = df_to_avg['dir_prof_films'].mean()
        
        df.at[i, 'compet_writ_rev'] = df_to_avg['writ_rev'].mean()
        df.at[i, 'compet_writ_prof'] = df_to_avg['writ_prof'].mean()
        df.at[i, 'compet_writ_films'] = df_to_avg['writ_films'].mean()
        df.at[i, 'compet_writ_prof_films'] = df_to_avg['writ_prof_films'].mean()
    
    return df

In [81]:
df = get_competitor_metrics(df, 'id')

In [89]:
df = df.dropna() #dropping films without competitors in the data

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3600 entries, 0 to 3798
Data columns (total 41 columns):
budget                    3600 non-null float64
id                        3600 non-null int64
release_date              3600 non-null datetime64[ns]
revenue                   3600 non-null float64
runtime                   3600 non-null float64
title                     3600 non-null object
genre                     3600 non-null object
profit                    3600 non-null float64
made_money                3600 non-null int64
cast_0                    3600 non-null object
cast_1                    3600 non-null object
cast_2                    3600 non-null object
cast_3                    3600 non-null object
cast_4                    3600 non-null object
director                  3600 non-null object
writer                    3600 non-null object
releases                  3600 non-null int64
cast_rev                  3600 non-null float64
cast_prof                 3600 non-nu

#### Cleaned data to csv file:

In [91]:
# df.to_csv(path_or_buf='/home/ec2-user/clean_data_v1.csv')

### ---side thing for j---

In [ ]:
j_info = info_df

In [ ]:
drop_cols = ['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path',
       'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'video', 'vote_average', 'vote_count',]

j_info = j_info.drop(drop_cols, axis=1)

In [ ]:
j_info = j_info.dropna()

In [ ]:
j_info = j_info.astype({"title": str, 
                          "genre": str})

In [ ]:
j_info = j_info.reset_index().drop('index', axis=1)

In [ ]:
# j_info.to_csv(path_or_buf='/home/ec2-user/titles_and_genres.csv')